# Computer Vision III: Detection, Segmentation and Tracking (CV3DST) GNN Exercise

In this exercise, we will first develop an extension of the ReID-based tracker we created in the previous exercise that will make it more robust to occlusions by allowing it to recover from missed detections. 

We will then implement a Message Passing Network from scratch, and we will use to build a model that will learn to combine position information and reid features to directly predict associations between past tracks and detections. We will use this model to create robust tracker. 

Your tasks are the following:
- Adapt the track management scheme of our ReIDTracker allow it to recover from missed detections.
- Implement a Message Passing Network from scratch to operate on bipartite graphs
- Implement the pairwise feature  computation to obtain features for our Message Passing Network
- Train the Message Passing Network and improve your tracker's IDF1 score


## Setup

### Download and extract project data to your Google Drive

1.   **Required**: Please follow all instructions of exercise 0 before running this notebook.
2.   Save this notebook to your Google Drive by clicking `Save a copy in Drive` from the `File` menu.
3.   Download [this](https://vision.in.tum.de/webshare/u/brasoand/cv3dst/cv3dst_gnn_exercise.zip) zip file to your desktop, extract it and upload it into the `Colab Notebooks` folder in your Google Drive.

#### Connect the notebook to your Google Drive

In [1]:
# from google.colab import drive

# drive.mount('/content/gdrive')

In [2]:
# root_dir = "gdrive/My Drive/Colab Notebooks/cv3dst_exercise/"
# gnn_root_dir = "gdrive/My Drive/Colab Notebooks/cv3dst_gnn_exercise/"

root_dir = "./cv3dst_exercise/"
gnn_root_dir = "./cv3dst_gnn_exercise/"

The `root_dir` path points to the directory and the content in your Google Drive.

In [3]:
# !ls "gdrive/My Drive/Colab Notebooks/cv3dst_gnn_exercise/data"

#### Install and import Python libraries

In [4]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

!pip install tqdm lap
!pip install https://github.com/timmeinhardt/py-motmetrics/archive/fix_pandas_deprecating_warnings.zip

  Using cached https://github.com/timmeinhardt/py-motmetrics/archive/fix_pandas_deprecating_warnings.zip


In [5]:
import os
import sys
sys.path.append(os.path.join(gnn_root_dir, 'src'))


import matplotlib.pyplot as plt
import numpy as np
import time
from tqdm.autonotebook import tqdm

import torch
from torch.utils.data import DataLoader

from tracker.data_track import MOT16Sequences
from tracker.tracker import Tracker, ReIDTracker
from tracker.utils import run_tracker, cosine_distance
from scipy.optimize import linear_sum_assignment as linear_assignment
import os.path as osp

import motmetrics as mm
mm.lap.default_solver = 'lap'

D:\Program Files\Conda\envs\i2dl\lib\site-packages\ipykernel_launcher.py:9: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  if __name__ == '__main__':


In [6]:
# !ls "gdrive/My Drive/Colab Notebooks/cv3dst_exercise/data/MOT16/train"
# !ls "gdrive/My Drive/Colab Notebooks/cv3dst_exercise/data/MOT16/test"

## Speed-Ups
In order to speed up training and inference runtimes, in this exercise we will be working with pre-computed detections and ReID embeddings. We ran the object detector we provided in Exercise 0 and applied to all frames. We also computed reid embeddings for all boxes in every frame of the dataset so that they don't need to be computed every time you run your tracker. This yields over 10x speed improvements. You will not have to work directly with the resulting files, as we have internally adapted the boilerplate code to work with them.

In [7]:
train_db = torch.load(osp.join(gnn_root_dir, 'data/preprocessed_data_train_2.pth'))

## Exercise Part 0 - Assignment's 1 ReIDHungarianTracker

We start by providing a sample solution of the ``ReIDTracker`` from Exercise 1.
It will serve as our baseline.


Recall that this tracker works by performing frame-to-frame bipartite matching between newly detected boxes and past tracks based on ReID distance. Whenever a past track cannot be matched, its killed. And whenever, a newly detected box cannot be match, it starts a new trajectory.

**NOTE**: We have modified the ``compute_distance`` function in ``data_association`` from last week to include a thresshold on ReID distance (if ReID distance >0.1, matching is not possible). This is important to prevent our tracker from reusing tracks for very dissimilar objects.


In [8]:
_UNMATCHED_COST=255
class ReIDHungarianTracker(ReIDTracker):
    def data_association(self, boxes, scores, pred_features):  
        """Refactored from previous implementation to split it onto distance computation and track management"""
        if self.tracks:
            track_boxes = torch.stack([t.box for t in self.tracks], axis=0)
            track_features = torch.stack([t.get_feature() for t in self.tracks], axis=0)
            
            distance = self.compute_distance_matrix(track_features, pred_features,
                                                    track_boxes, boxes, metric_fn=cosine_distance)

            # Perform Hungarian matching.
            row_idx, col_idx = linear_assignment(distance)            
            self.update_tracks(row_idx, col_idx,distance, boxes, scores, pred_features)

            
        else:
            # No tracks exist.
            self.add(boxes, scores, pred_features)
        
    def update_tracks(self, row_idx, col_idx, distance, boxes, scores, pred_features):
        """Updates existing tracks and removes unmatched tracks.
           Reminder: If the costs are equal to _UNMATCHED_COST, it's not a 
           match. 
        """
        track_ids = [t.id for t in self.tracks]

        unmatched_track_ids = []
        seen_track_ids = []
        seen_box_idx = []
        for track_idx, box_idx in zip(row_idx, col_idx):
            costs = distance[track_idx, box_idx] 
            internal_track_id = track_ids[track_idx]
            seen_track_ids.append(internal_track_id)
            if costs == _UNMATCHED_COST:
                unmatched_track_ids.append(internal_track_id)
            else:
                self.tracks[track_idx].box = boxes[box_idx]
                self.tracks[track_idx].add_feature(pred_features[box_idx])
                seen_box_idx.append(box_idx)

        unseen_track_ids = set(track_ids) - set(seen_track_ids)
        unmatched_track_ids.extend(list(unseen_track_ids))
        self.tracks = [t for t in self.tracks
                       if t.id not in unmatched_track_ids]


        # Add new tracks.
        new_boxes_idx = set(range(len(boxes))) - set(seen_box_idx)
        new_boxes = [boxes[i] for i in new_boxes_idx]
        new_scores = [scores[i] for i in new_boxes_idx]
        new_features = [pred_features[i] for i in new_boxes_idx]
        self.add(new_boxes, new_scores, new_features)


In [9]:
val_sequences = MOT16Sequences('MOT16-reid', root_dir = osp.join(root_dir, 'data/MOT16'), vis_threshold=0.)

In [10]:
tracker = ReIDHungarianTracker(None)
run_tracker(val_sequences, db=train_db, tracker=tracker, output_dir=None)

Tracking: MOT16-02
Tracks found: 314
Runtime for MOT16-02: 4.1 s.
Tracking: MOT16-05
Tracks found: 295
Runtime for MOT16-05: 1.0 s.
Tracking: MOT16-09
Tracks found: 87
Runtime for MOT16-09: 0.8 s.
Tracking: MOT16-11
Tracks found: 187
Runtime for MOT16-11: 1.8 s.
Runtime for all sequences: 7.7 s.
          IDF1   IDP   IDR  Rcll  Prcn  GT  MT  PT ML   FP    FN IDs   FM  MOTA  MOTP
MOT16-02 41.6% 59.1% 32.1% 52.2% 96.1%  62  11  39 12  390  8873 203  216 49.1% 0.096
MOT16-05 57.9% 68.5% 50.1% 68.8% 94.0% 133  56  65 12  305  2156 176  146 61.9% 0.142
MOT16-09 52.2% 64.6% 43.8% 66.3% 97.7%  26  13  12  1   82  1793  72   79 63.4% 0.083
MOT16-11 63.4% 69.9% 58.0% 80.2% 96.6%  75  44  24  7  266  1871  88   90 76.4% 0.083
OVERALL  51.6% 64.8% 42.8% 63.5% 96.1% 296 124 140 32 1043 14693 539  531 59.6% 0.099


,idf1,idp,idr,recall,precision,num_unique_objects,mostly_tracked,partially_tracked,mostly_lost,num_false_positives,num_misses,num_switches,num_fragmentations,mota,motp
MOT16-02,0.416193,0.591008,0.321188,0.522469,0.961378,62,11,39,12,390,8873,203,216,0.490555,0.095583
MOT16-05,0.578820,0.684564,0.501373,0.688304,0.939795,133,56,65,12,305,2156,176,146,0.618765,0.141840
MOT16-09,0.522430,0.646099,0.438498,0.663286,0.977310,26,13,12,1,82,1793,72,79,0.634366,0.082864
MOT16-11,0.634042,0.699017,0.580119,0.801717,0.966032,75,44,24,7,266,1871,88,90,0.764201,0.082746
OVERALL,0.515792,0.648089,0.428351,0.635038,0.960803,296,124,140,32,1043,14693,539,531,0.595743,0.098641


## Exercise Part I - Long-Term ReID Tracker


The tracker above has an obvious limitation: whenever a track cannot be matched with the detections of a given frame the track will be killed. This means that if our detector misses an object in a single frame (due to e.g. occlusion), we will not be able to recover that track, and we will start a new one. 

To fix this issue, we would like to allow our tracker to maintain tracks that are not matched during data association. We will refer to these tracks as **inactive**. During data association, we will try to match the detected boxes for the current frame to both tracks that are active (i.e. tracks that we were able to match in the previous frame) as well as those that are inactive. Therefore, if a detector misses an object in a frame and the object reappears after a few frames, we will still be able to match it to its corresponding track, instead of creating a new one.

In order to adapt our tracker to have this behavior, we will use the `inactive` attribute from the `track` class (see `tracker/tracker.py`. This attribute will be assigned an integer indicating for how many frames a track has remained unmatched. Whenever we are able to match the track `t`, we will set `t.inactive=0` and, naturally, when tracks are initialized, the class constructor sets `inactive=0`. 

Your job is to maintain the `inactive` attribute of all tracks being kept by tracker so that its value represents the number of frames for which the track has been unmatched. Additionally, we introduce a `patience` parameter. Whenever a track has been inactive for more than `inactive` frames. it will need to be killed.

In [11]:
class LongTermReIDHungarianTracker(ReIDHungarianTracker):
    def __init__(self, patience, *args, **kwargs):
        """ Add a patience parameter"""
        self.patience=patience
        super().__init__(*args, **kwargs)

    def update_results(self):
        """Only store boxes for tracks that are active"""
        for t in self.tracks:
            if t.id not in self.results.keys():
                self.results[t.id] = {}
            if t.inactive == 0: # Only change
                self.results[t.id][self.im_index] = np.concatenate([t.box.cpu().numpy(), np.array([t.score])])

        self.im_index += 1        
        
    def update_tracks(self, row_idx, col_idx, distance, boxes, scores, pred_features):
        track_ids = [t.id for t in self.tracks]

        unmatched_track_ids = []
        seen_track_ids = []
        seen_box_idx = []
        for track_idx, box_idx in zip(row_idx, col_idx):
            costs = distance[track_idx, box_idx] 
            internal_track_id = track_ids[track_idx]
            seen_track_ids.append(internal_track_id)
            if costs == _UNMATCHED_COST:
                unmatched_track_ids.append(internal_track_id)

            else:
                self.tracks[track_idx].box = boxes[box_idx]
                self.tracks[track_idx].add_feature(pred_features[box_idx])
                
                # Note: the track is matched, therefore, inactive is set to 0
                self.tracks[track_idx].inactive=0
                seen_box_idx.append(box_idx)
                

        unseen_track_ids = set(track_ids) - set(seen_track_ids)
        unmatched_track_ids.extend(list(unseen_track_ids))
        ##################
        ### TODO starts
        ##################
        
        # Update the `inactive` attribute for those tracks that have
        # not been matched. kill those for which the inactive parameter 
        # is > self.patience
        new_tracks = [];
        for t in self.tracks:
            if t.id in unmatched_track_ids:
                t.inactive += 1
            if t.inactive <= self.patience:
                new_tracks.append(t)

        self.tracks = new_tracks
        ##################
        ### TODO ends
        ##################        
        
        new_boxes_idx = set(range(len(boxes))) - set(seen_box_idx)
        new_boxes = [boxes[i] for i in new_boxes_idx]
        new_scores = [scores[i] for i in new_boxes_idx]
        new_features = [pred_features[i] for i in new_boxes_idx]
        self.add(new_boxes, new_scores, new_features)

In [12]:
tracker = LongTermReIDHungarianTracker(patience=20, obj_detect=None)
run_tracker(val_sequences, db=train_db, tracker=tracker, output_dir=None)

Tracking: MOT16-02
Tracks found: 130
Runtime for MOT16-02: 4.2 s.
Tracking: MOT16-05
Tracks found: 155
Runtime for MOT16-05: 1.3 s.
Tracking: MOT16-09
Tracks found: 51
Runtime for MOT16-09: 0.8 s.
Tracking: MOT16-11
Tracks found: 91
Runtime for MOT16-11: 1.8 s.
Runtime for all sequences: 8.0 s.
          IDF1   IDP   IDR  Rcll  Prcn  GT  MT  PT ML   FP    FN IDs   FM  MOTA  MOTP
MOT16-02 47.2% 67.0% 36.4% 52.2% 96.1%  62  11  38 13  390  8873 142  220 49.4% 0.095
MOT16-05 62.5% 74.0% 54.2% 68.8% 94.0% 133  56  65 12  305  2156 119  149 62.7% 0.142
MOT16-09 56.6% 70.0% 47.5% 66.3% 97.7%  26  12  13  1   82  1793  41   83 64.0% 0.085
MOT16-11 69.4% 76.5% 63.5% 80.2% 96.6%  75  44  25  6  266  1871  48   90 76.8% 0.083
OVERALL  56.9% 71.5% 47.3% 63.5% 96.1% 296 123 141 32 1043 14693 350  542 60.0% 0.099


,idf1,idp,idr,recall,precision,num_unique_objects,mostly_tracked,partially_tracked,mostly_lost,num_false_positives,num_misses,num_switches,num_fragmentations,mota,motp
MOT16-02,0.472122,0.670430,0.364351,0.522469,0.961378,62,11,38,13,390,8873,142,220,0.493838,0.094828
MOT16-05,0.625386,0.739637,0.541709,0.688304,0.939795,133,56,65,12,305,2156,119,149,0.627006,0.141822
MOT16-09,0.566059,0.700055,0.475117,0.663286,0.977310,26,12,13,1,82,1793,41,83,0.640188,0.085351
MOT16-11,0.693693,0.764781,0.634697,0.801717,0.966032,75,44,25,6,266,1871,48,90,0.768440,0.082838
OVERALL,0.569361,0.715397,0.472838,0.635038,0.960803,296,123,141,32,1043,14693,350,542,0.600437,0.098722


## Exercise Part II - Building a tracker based on Neural Message Passing

Our ``LongTermReIDHungarianTracker`` is still limited when compared to current modern trackers. 

Firstly, it relies solely on appearance to predict similarity scores between objectes. This can be problematic whenever appearance alone may not discriminative, and it'd be best to also take into account object position and size attributes. Secondly, our tracker can only account for pairwise similarities among objects. Ideally, we would like it to also consider higher-order information.

To address these limitations. We will now build a tracker that will combine both apperance and position information with a Message Passing Neural Network, inspired by the approach presented in [Learning a Neural Solver for Multiple Object Tracking, CVPR 2020](https://arxiv.org/abs/1912.07515)

The overall idea will be to build, for every tracking step, a bipartite graph containing two sets of nodes: past tracks, and detections in the current frame. We will initialize node features with ReID embeddings, and edge features with relative position features and ReID distance. We will use an MPN to refine these edge embeddings. The learning task will be to classify the edge embeddings in this graph, which is equivalent to predicting the entries of our data association similarity matrix.


### Building an MPN for Bipartite Graphs

We will first build a Neural Message Passing layer based on the Graph Networks framework introduced in [Relational inductive biases, deep learning, and graph networks, arXiv 2020](https://arxiv.org/abs/1806.01261), as explained in the *A More General Framework* slides of [Lecture 5](https://www.moodle.tum.de/pluginfile.php/2928927/mod_resource/content/1/5.MOT2.pdf) (slides 70 to 75).

We will be using a bipartite graph, i.e., we will have two sets of nodes $A$ (past tracks), and $B$ (detections), and our set of edges will be $A\times B$. That is, we will connect every pair of past tracks and detections.

We will have initial node features (i.e. reid embeddings) matrices: $X_A$ and $X_B$ and an initial edge features tensor $E$.

$X_A$ and $X_B$ have shape $|A|\times \text{node_dim}$ and $|B|\times \text{node_dim}$, respectively.

$E$ has shape $|A| \times |B| \times \text{edge_dim}$. Its $(i, j)$ entry contains the edge features of node $i$ in $A$ and node $j$ in $B$.

With the given layer, we will produce new node feature matrices $X_A'$ and $X_B'$ and edge features $E'$ with the same dimensions. 
Please refer to the formulas in the slides and figure how to apply them in this setting.

You are asked to implement both the node and edge update steps in the class below

**NOTE 1**: Working with a bipartite graph allows us to vectorize all operations in the formulas in a straightforward manner (keep in mind that we store edge features in a matrix). Given a node in $A$, it is connected to all nodes in $B$.

**NOTE 2**: You do not need to care about batching several graphs. This implementation will only work with a single graph at a time.

In [13]:
from torch import nn

class BipartiteNeuralMessagePassingLayer(nn.Module):    
    def __init__(self, node_dim, edge_dim, dropout=0.):
        super().__init__()

        edge_in_dim  = 2*node_dim + 2*edge_dim # 2*edge_dim since we always concatenate initial edge features
        self.edge_mlp = nn.Sequential(*[nn.Linear(edge_in_dim, edge_dim), nn.ReLU(), nn.Dropout(dropout), 
                                    nn.Linear(edge_dim, edge_dim), nn.ReLU(), nn.Dropout(dropout)])

        node_in_dim  = node_dim + edge_dim
        self.node_mlp = nn.Sequential(*[nn.Linear(node_in_dim, node_dim), nn.ReLU(), nn.Dropout(dropout),  
                                        nn.Linear(node_dim, node_dim), nn.ReLU(), nn.Dropout(dropout)])

    def edge_update(self, edge_embeds, nodes_a_embeds, nodes_b_embeds):
        """
        Node-to-edge updates, as descibed in slide 71, lecture 5.
        Args:
            edge_embeds: torch.Tensor with shape (|A|, |B|, 2 x edge_dim) 
            nodes_a_embeds: torch.Tensor with shape (|A|, node_dim)
            nodes_a_embeds: torch.Tensor with shape (|B|, node_dim)
            
        returns:
            updated_edge_feats = torch.Tensor with shape (|A|, |B|, edge_dim) 
        """
        
        n_nodes_a, n_nodes_b, _  = edge_embeds.shape
        _, node_dim = nodes_a_embeds.shape
        
        ########################
        #### TODO starts
        ########################
        
        nodes_a_embeds_repeated =  nodes_a_embeds.reshape((n_nodes_a, 1, node_dim)).repeat_interleave(n_nodes_b, axis = 1)  # |A|, |B|, node_dim
        nodes_b_embeds_repeated =  nodes_b_embeds.reshape((1, n_nodes_b, node_dim)).repeat_interleave(n_nodes_a, axis = 0)  # |A|, |B|, node_dim
        edge_in = torch.cat((nodes_a_embeds_repeated, edge_embeds, nodes_b_embeds_repeated), 2) # |A|, |B|, 2*edge_dim + 2*node_dim

        ########################
        #### TODO ends
        ########################
        
        
        return self.edge_mlp(edge_in)

    def node_update(self, edge_embeds, nodes_a_embeds, nodes_b_embeds):
        """
        Edge-to-node updates, as descibed in slide 75, lecture 5.

        Args:
            edge_embeds: torch.Tensor with shape (|A|, |B|, edge_dim) 
            nodes_a_embeds: torch.Tensor with shape (|A|, node_dim)
            nodes_b_embeds: torch.Tensor with shape (|B|, node_dim)
            
        returns:
            tuple(
                updated_nodes_a_embeds: torch.Tensor with shape (|A|, node_dim),
                updated_nodes_b_embeds: torch.Tensor with shape (|B|, node_dim)
                )
        """
        
        ########################
        #### TODO starts
        ########################
        nodes_a, nodes_b, edge_dim = edge_embeds.shape

        # NOTE: Use 'sum' as aggregation function
        a_edges = torch.sum(edge_embeds, 1).reshape((nodes_a, edge_dim))
        b_edges = torch.sum(edge_embeds, 0).reshape((nodes_b, edge_dim))
        
        nodes_a_in = torch.cat((nodes_a_embeds, a_edges), 1)
        nodes_b_in = torch.cat((nodes_b_embeds, b_edges), 1)

        ########################
        #### TODO ends
        ########################

        nodes_a = self.node_mlp(nodes_a_in)
        nodes_b = self.node_mlp(nodes_b_in)

        return nodes_a, nodes_b

    def forward(self, edge_embeds, nodes_a_embeds, nodes_b_embeds):
        edge_embeds_latent = self.edge_update(edge_embeds, nodes_a_embeds, nodes_b_embeds)
        nodes_a_latent, nodes_b_latent = self.node_update(edge_embeds_latent, nodes_a_embeds, nodes_b_embeds)

        return edge_embeds_latent, nodes_a_latent, nodes_b_latent

## Building the entire network to predict similarities
We now build the network that generates initial node and edge features, performs neural message passing, and classifies edges in order to produce the final costs that we will use for data association.

You need to implement the method that computes the initial edge features. You can can follow [1] and, given a two bounding boxes $(x_i, y_i, w_i, h_i)$ and  $(x_j, y_j, w_j, h_j)$ and timestamps $t_i$ and $t_j$, compute an initial 5-dimensional edge feature vector as:
$$ E_(i, j) = \left (\frac{2(x_j - x_i)}{h_i + h_j}, \frac{2(y_j - y_i)}{h_i + h_j}, \log{\frac{h_i}{h_j}}, \log{\frac{w_i}{w_j}}, t_j - t_i \right )$$


Feel free to engineer your own features (e.g. use IoU, etc.)

In [14]:
from torch.nn import functional as F
class AssignmentSimilarityNet(nn.Module):
    def __init__(self, reid_network, node_dim, edge_dim, reid_dim, edges_in_dim, num_steps, dropout=0.):
        super().__init__()
        self.reid_network = reid_network
        self.graph_net = BipartiteNeuralMessagePassingLayer(node_dim=node_dim, edge_dim=edge_dim, dropout=dropout)
        self.num_steps = num_steps
        self.cnn_linear = nn.Linear(reid_dim, node_dim)
        self.edge_in_mlp = nn.Sequential(*[nn.Linear(edges_in_dim, edge_dim), nn.ReLU(), nn.Dropout(dropout), nn.Linear(edge_dim, edge_dim), nn.ReLU(),nn.Dropout(dropout)])
        self.classifier = nn.Sequential(*[nn.Linear(edge_dim, edge_dim), nn.ReLU(), nn.Linear(edge_dim, 1)])
        
    
    def compute_edge_feats(self, track_coords, current_coords, track_t, curr_t):    
        """
        Computes initial edge feature tensor

        Args:
            track_coords: track's frame box coordinates, given by top-left and bottom-right coordinates
                          torch.Tensor with shape (num_tracks, 4)
            current_coords: current frame box coordinates, given by top-left and bottom-right coordinates
                            has shape (num_boxes, 4)
                          
            track_t: track's timestamps, torch.Tensor with with shape (num_tracks, )
            curr_t: current frame's timestamps, torch.Tensor withwith shape (num_boxes,)        
            
        
        Returns:
            tensor with shape (num_trakcs, num_boxes, 5) containing pairwise
            position and time difference features 
        """

        ########################
        #### TODO starts
        ########################
        n_a, _ = track_coords.shape
        n_b, _ = current_coords.shape
        
        # NOTE 1: we recommend you to use box centers to compute distances
        # in the x and y coordinates.

        # NOTE 2: Check out the  code inside train_one_epoch function and 
        # LongTrackTrainingDataset class a few cells below to debug this


        track_coords_reshaped = track_coords.reshape((n_a, 1, 4)).repeat_interleave(n_b, axis = 1)
        current_coords_reshaped = current_coords.reshape((1, n_b, 4)).repeat_interleave(n_a, axis = 0)
        track_t_reshaped = track_t.reshape((n_a, 1, 1)).repeat_interleave(n_b, axis = 1)
        curr_t_reshaped = curr_t.reshape((1, n_b, 1)).repeat_interleave(n_a, axis = 0)

        edge_feats = torch.zeros((n_a,n_b,5))
        edge_feats[:, :, 0] = 2*(current_coords_reshaped[:,:,0]-track_coords_reshaped[:,:,0])/(track_coords_reshaped[:,:,3]+current_coords_reshaped[:,:,3])
        edge_feats[:, :, 1] = 2*(current_coords_reshaped[:,:,1]-track_coords_reshaped[:,:,1])/(track_coords_reshaped[:,:,3]+current_coords_reshaped[:,:,3])
        edge_feats[:, :, 2] = torch.log(track_coords_reshaped[:,:,3]/current_coords_reshaped[:,:,3])
        edge_feats[:, :, 3] = torch.log(track_coords_reshaped[:,:,2]/current_coords_reshaped[:,:,2])
        edge_feats[:, :, 4] = curr_t_reshaped[:,:,0]-track_t_reshaped[:,:,0]

        ########################
        #### TODO ends
        ########################

        return edge_feats # has shape (num_trakcs, num_boxes, 5)


    def forward(self, track_app, current_app, track_coords, current_coords, track_t, curr_t):
        """
        Args:
            track_app: track's reid embeddings, torch.Tensor with shape (num_tracks, 512)
            current_app: current frame detections' reid embeddings, torch.Tensor with shape (num_boxes, 512)
            track_coords: track's frame box coordinates, given by top-left and bottom-right coordinates
                          torch.Tensor with shape (num_tracks, 4)
            current_coords: current frame box coordinates, given by top-left and bottom-right coordinates
                            has shape (num_boxes, 4)
                          
            track_t: track's timestamps, torch.Tensor with with shape (num_tracks, )
            curr_t: current frame's timestamps, torch.Tensor withwith shape (num_boxes,)
            
        Returns:
            classified edges: torch.Tensor with shape (num_steps, num_tracks, num_boxes),
                             containing at entry (step, i, j) the unnormalized probability that track i and 
                             detection j are a match, according to the classifier at the given neural message passing step
        """
        
        # Get initial edge embeddings to
        dist_reid = cosine_distance(track_app, current_app)
        pos_edge_feats = self.compute_edge_feats(track_coords, current_coords, track_t, curr_t)

        edge_feats = torch.cat((pos_edge_feats.cuda(), dist_reid.unsqueeze(-1)), dim=-1)
        edge_embeds = self.edge_in_mlp(edge_feats)
        initial_edge_embeds = edge_embeds.clone()

        # Get initial node embeddings, reduce dimensionality from 512 to node_dim
        track_embeds = F.relu(self.cnn_linear(track_app))
        curr_embeds =F.relu(self.cnn_linear(current_app))

        classified_edges = []
        for _ in range(self.num_steps):
            edge_embeds = torch.cat((edge_embeds, initial_edge_embeds), dim=-1)            
            edge_embeds, track_embeds, curr_embeds = self.graph_net(edge_embeds=edge_embeds, 
                                                                    nodes_a_embeds=track_embeds, 
                                                                    nodes_b_embeds=curr_embeds)

            classified_edges.append(self.classifier(edge_embeds))

        return torch.stack(classified_edges).squeeze(-1)

## Putting everything together

Finally, we incorporate our ``AssignmentSimilarityNet`` into our tracker. We can keep everything as in ``LongTermReIDHungarianTracker`` except for the distance computation, which is now directly obtained via a forward pass through AssignmentSimilarityNet.

In [15]:
_UNMATCHED_COST=255
class MPNTracker(LongTermReIDHungarianTracker):
    def __init__(self, assign_net, *args, **kwargs):
        self.assign_net = assign_net
        super().__init__(*args, **kwargs)
        
    def data_association(self, boxes, scores, pred_features):  
        if self.tracks:  
            track_boxes = torch.stack([t.box for t in self.tracks], axis=0).cuda()
            track_features = torch.stack([t.get_feature() for t in self.tracks], axis=0).cuda()
            
            # Hacky way to recover the timestamps of boxes and tracks
            curr_t = self.im_index * torch.ones((pred_features.shape[0],)).cuda()
            track_t = torch.as_tensor([self.im_index - t.inactive - 1 for t in self.tracks]).cuda()

            ########################
            #### TODO starts
            ########################
            
            # Do a forward pass through self.assign_net to obtain our costs.
            # Note: self.assign_net will return unnormalized probabilities. 
            # Make sure to apply the sigmoid function to them!

            pred_sim = self.assign_net(track_features, pred_features.cuda(), track_boxes, boxes.cuda(), track_t, curr_t)
            pred_sim = torch.sigmoid(pred_sim)

            ########################
            #### TODO ends
            ########################

            pred_sim = pred_sim[-1].cpu().numpy()  # Use predictions at last message passing step
            distance = (1- pred_sim)
            
            # Do not allow matches when sim < 0.5, to avoid low-confident associations
            distance = np.where(pred_sim < 0.5, _UNMATCHED_COST, distance)

            # Perform Hungarian matching.
            row_idx, col_idx = linear_assignment(distance)            
            self.update_tracks(row_idx, col_idx,distance, boxes, scores, pred_features)

            
        else:
            # No tracks exist.
            self.add(boxes, scores, pred_features)

## Training and evaluating our model

We provide all boilerplate code for training our neural message passing based
tracker, as well as evaluating. 

Under the hood, we are sampling frames randomly from our training sequences, and then sampling boxes from past frames as past_tracks to generate our 
training
data. Check out `LongTrackTrainingDataset` for details.

We train the model with a weighted cross-entropy loss
to account for the class imbalance. Check out `train_one_epoch` if you're 
interested.

No need to write any code from your side here!


In [18]:
from gnn.dataset import LongTrackTrainingDataset
from torch.utils.data import DataLoader
from gnn.trainer import train_one_epoch

MAX_PATIENCE = 20
MAX_EPOCHS = 15
EVAL_FREQ = 1


# Define our model, and init 
assign_net = AssignmentSimilarityNet(reid_network=None, # Not needed since we work with precomputed features
                                     node_dim=32, 
                                     edge_dim=64, 
                                     reid_dim=512, 
                                     edges_in_dim=6, 
                                     num_steps=10).cuda()

# We only keep two sequences for validation. You can
dataset = LongTrackTrainingDataset(dataset='MOT16-train_wo_val2', 
                                   db=train_db, 
                                   root_dir= osp.join(root_dir, 'data/MOT16'),
                                   max_past_frames = MAX_PATIENCE,
                                   vis_threshold=0.25)

data_loader = DataLoader(dataset, batch_size=8, collate_fn = lambda x: x, 
                         shuffle=True, num_workers=0, drop_last=True)
device = torch.device('cuda')
optimizer = torch.optim.Adam(assign_net.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5)

We only leave 2 sequences for validation in order to maximize 
the amount of training data. For your convenience, here are the
 LongTermReIDTracker results on them. Your validation IDF1 scores should show an improvement of ~0.5 over them.
```

          IDF1   IDP   IDR  Rcll  Prcn  GT MT PT ML  FP    FN IDs   FM  MOTA  MOTP
MOT16-02 47.2% 67.0% 36.4% 52.2% 96.1%  62 11 38 13 390  8873 142  220 49.4% 0.095
MOT16-11 69.4% 76.5% 63.5% 80.2% 96.6%  75 44 25  6 266  1871  48   90 76.8% 0.083
OVERALL  55.5% 71.2% 45.5% 61.7% 96.3% 137 55 63 19 656 10744 190  310 58.6% 0.090
```



Let's start training!

Note that we have observed quite a lot of noise in validation scores among epochs and runs. This can be explained due to the small size of our training and
validation sets, that's why we perform early stopping to obtain the best performing model on validation. 

In [19]:
best_idf1 = 0.
for epoch in range(1, MAX_EPOCHS + 1):
    print(f"-------- EPOCH {epoch:2d} --------")
    train_one_epoch(model = assign_net, data_loader=data_loader, optimizer=optimizer, print_freq=100)
    scheduler.step()

    if epoch % EVAL_FREQ == 0:
        tracker =  MPNTracker(assign_net=assign_net.eval(), obj_detect=None, patience=MAX_PATIENCE)
        val_sequences = MOT16Sequences('MOT16-val2', osp.join(root_dir, 'data/MOT16'), vis_threshold=0.)
        res = run_tracker(val_sequences, db=train_db, tracker=tracker, output_dir=None)
        idf1 = res.loc['OVERALL']['idf1']
        if idf1 > best_idf1:
            best_idf1 = idf1
            torch.save(assign_net.state_dict(), osp.join(root_dir, 'output', 'best_ckpt.pth'))
        

-------- EPOCH  1 --------


Iter 100. Loss: 0.730. Accuracy: 0.950. Recall: 0.670. Precision: 0.509
Iter 200. Loss: 0.098. Accuracy: 0.994. Recall: 0.986. Precision: 0.915
Iter 300. Loss: 0.044. Accuracy: 0.998. Recall: 0.995. Precision: 0.972
Iter 400. Loss: 0.046. Accuracy: 0.998. Recall: 0.991. Precision: 0.974

Tracking: MOT16-02
Tracks found: 97
Runtime for MOT16-02: 9.1 s.
Tracking: MOT16-11
Tracks found: 83
Runtime for MOT16-11: 10.1 s.
Runtime for all sequences: 19.2 s.
          IDF1   IDP   IDR  Rcll  Prcn  GT MT PT ML  FP    FN IDs   FM  MOTA  MOTP
MOT16-02 46.7% 66.3% 36.0% 52.2% 96.1%  62 11 38 13 390  8873 122  216 49.5% 0.095
MOT16-11 68.2% 75.2% 62.4% 80.2% 96.6%  75 44 24  7 266  1871  37   90 77.0% 0.083
OVERALL  54.8% 70.2% 44.9% 61.7% 96.3% 137 55 62 20 656 10744 159  306 58.7% 0.089
-------- EPOCH  2 --------


Iter 100. Loss: 0.027. Accuracy: 0.998. Recall: 0.996. Precision: 0.984
Iter 200. Loss: 0.020. Accuracy: 0.999. Recall: 0.998. Precision: 0.987
Iter 300. Loss: 0.029. Accuracy: 0.998. Recall: 0.997. Precision: 0.984
Iter 400. Loss: 0.015. Accuracy: 0.999. Recall: 0.998. Precision: 0.989

Tracking: MOT16-02
Tracks found: 96
Runtime for MOT16-02: 9.2 s.
Tracking: MOT16-11
Tracks found: 96
Runtime for MOT16-11: 10.4 s.
Runtime for all sequences: 19.6 s.
          IDF1   IDP   IDR  Rcll  Prcn  GT MT PT ML  FP    FN IDs   FM  MOTA  MOTP
MOT16-02 44.6% 63.4% 34.4% 52.2% 96.1%  62 11 38 13 390  8873 118  217 49.5% 0.094
MOT16-11 67.3% 74.2% 61.6% 80.2% 96.6%  75 44 24  7 266  1871  42   90 76.9% 0.083
OVERALL  53.1% 68.1% 43.6% 61.7% 96.3% 137 55 62 20 656 10744 160  307 58.7% 0.089
-------- EPOCH  3 --------


Iter 100. Loss: 0.033. Accuracy: 0.998. Recall: 0.996. Precision: 0.980
Iter 200. Loss: 0.027. Accuracy: 0.998. Recall: 0.996. Precision: 0.980
Iter 300. Loss: 0.020. Accuracy: 0.999. Recall: 0.997. Precision: 0.987
Iter 400. Loss: 0.017. Accuracy: 0.999. Recall: 0.998. Precision: 0.985

Tracking: MOT16-02
Tracks found: 65
Runtime for MOT16-02: 9.8 s.
Tracking: MOT16-11
Tracks found: 69
Runtime for MOT16-11: 10.5 s.
Runtime for all sequences: 20.3 s.
          IDF1   IDP   IDR  Rcll  Prcn  GT MT PT ML  FP    FN IDs   FM  MOTA  MOTP
MOT16-02 39.4% 55.9% 30.4% 52.2% 96.1%  62 11 38 13 390  8873 121  216 49.5% 0.095
MOT16-11 64.4% 71.1% 59.0% 80.2% 96.6%  75 44 24  7 266  1871  38   89 76.9% 0.083
OVERALL  48.8% 62.5% 40.0% 61.7% 96.3% 137 55 62 20 656 10744 159  305 58.7% 0.089
-------- EPOCH  4 --------


Iter 100. Loss: 0.017. Accuracy: 0.998. Recall: 0.998. Precision: 0.982
Iter 200. Loss: 0.015. Accuracy: 0.999. Recall: 0.998. Precision: 0.986
Iter 300. Loss: 0.020. Accuracy: 0.998. Recall: 0.997. Precision: 0.977
Iter 400. Loss: 0.019. Accuracy: 0.999. Recall: 0.998. Precision: 0.984

Tracking: MOT16-02
Tracks found: 82
Runtime for MOT16-02: 9.2 s.
Tracking: MOT16-11
Tracks found: 81
Runtime for MOT16-11: 10.3 s.
Runtime for all sequences: 19.6 s.
          IDF1   IDP   IDR  Rcll  Prcn  GT MT PT ML  FP    FN IDs   FM  MOTA  MOTP
MOT16-02 45.4% 64.5% 35.0% 52.2% 96.1%  62 11 38 13 390  8873 125  216 49.5% 0.095
MOT16-11 68.7% 75.7% 62.9% 80.2% 96.6%  75 44 24  7 266  1871  33   90 77.0% 0.083
OVERALL  54.2% 69.4% 44.4% 61.7% 96.3% 137 55 62 20 656 10744 158  306 58.7% 0.089
-------- EPOCH  5 --------


...

Iter 100. Loss: 0.013. Accuracy: 0.998. Recall: 0.998. Precision: 0.982
Iter 200. Loss: 0.009. Accuracy: 0.999. Recall: 1.000. Precision: 0.991
Iter 300. Loss: 0.013. Accuracy: 0.999. Recall: 0.998. Precision: 0.988
Iter 400. Loss: 0.012. Accuracy: 0.998. Recall: 0.999. Precision: 0.984

Tracking: MOT16-02
Tracks found: 84
Runtime for MOT16-02: 11.0 s.
Tracking: MOT16-11
Tracks found: 80
Runtime for MOT16-11: 11.0 s.
Runtime for all sequences: 22.0 s.
          IDF1   IDP   IDR  Rcll  Prcn  GT MT PT ML  FP    FN IDs   FM  MOTA  MOTP
MOT16-02 47.2% 67.1% 36.5% 52.2% 96.1%  62 11 38 13 390  8873 114  214 49.5% 0.095
MOT16-11 68.5% 75.5% 62.6% 80.2% 96.6%  75 44 24  7 266  1871  31   89 77.0% 0.083
OVERALL  55.2% 70.7% 45.3% 61.7% 96.3% 137 55 62 20 656 10744 145  303 58.8% 0.089
-------- EPOCH 12 --------


Iter 100. Loss: 0.011. Accuracy: 0.999. Recall: 1.000. Precision: 0.986
Iter 200. Loss: 0.011. Accuracy: 0.999. Recall: 0.998. Precision: 0.986
Iter 300. Loss: 0.012. Accuracy: 0.999. Recall: 0.998. Precision: 0.985
Iter 400. Loss: 0.012. Accuracy: 0.999. Recall: 0.999. Precision: 0.985

Tracking: MOT16-02
Tracks found: 86
Runtime for MOT16-02: 9.2 s.
Tracking: MOT16-11
Tracks found: 80
Runtime for MOT16-11: 10.7 s.
Runtime for all sequences: 19.9 s.
          IDF1   IDP   IDR  Rcll  Prcn  GT MT PT ML  FP    FN IDs   FM  MOTA  MOTP
MOT16-02 47.1% 66.9% 36.4% 52.2% 96.1%  62 11 38 13 390  8873 114  214 49.5% 0.095
MOT16-11 68.5% 75.5% 62.6% 80.2% 96.6%  75 44 24  7 266  1871  31   89 77.0% 0.083
OVERALL  55.1% 70.6% 45.2% 61.7% 96.3% 137 55 62 20 656 10744 145  303 58.8% 0.089
-------- EPOCH 13 --------


Iter 100. Loss: 0.013. Accuracy: 0.999. Recall: 0.998. Precision: 0.987
Iter 200. Loss: 0.010. Accuracy: 0.999. Recall: 0.999. Precision: 0.986
Iter 300. Loss: 0.013. Accuracy: 0.999. Recall: 0.999. Precision: 0.985
Iter 400. Loss: 0.013. Accuracy: 0.998. Recall: 0.998. Precision: 0.984

Tracking: MOT16-02
Tracks found: 86
Runtime for MOT16-02: 9.3 s.
Tracking: MOT16-11
Tracks found: 80
Runtime for MOT16-11: 10.6 s.
Runtime for all sequences: 19.9 s.
          IDF1   IDP   IDR  Rcll  Prcn  GT MT PT ML  FP    FN IDs   FM  MOTA  MOTP
MOT16-02 47.0% 66.7% 36.2% 52.2% 96.1%  62 11 38 13 390  8873 114  214 49.5% 0.095
MOT16-11 68.5% 75.5% 62.6% 80.2% 96.6%  75 44 24  7 266  1871  31   89 77.0% 0.083
OVERALL  55.0% 70.5% 45.1% 61.7% 96.3% 137 55 62 20 656 10744 145  303 58.8% 0.089
-------- EPOCH 14 --------


Iter 100. Loss: 0.010. Accuracy: 0.999. Recall: 0.999. Precision: 0.988
Iter 200. Loss: 0.013. Accuracy: 0.999. Recall: 0.998. Precision: 0.984
Iter 300. Loss: 0.012. Accuracy: 0.999. Recall: 0.999. Precision: 0.985
Iter 400. Loss: 0.010. Accuracy: 0.999. Recall: 0.999. Precision: 0.988

Tracking: MOT16-02
Tracks found: 86
Runtime for MOT16-02: 9.1 s.
Tracking: MOT16-11
Tracks found: 81
Runtime for MOT16-11: 10.1 s.
Runtime for all sequences: 19.2 s.
          IDF1   IDP   IDR  Rcll  Prcn  GT MT PT ML  FP    FN IDs   FM  MOTA  MOTP
MOT16-02 48.4% 68.8% 37.4% 52.2% 96.1%  62 11 38 13 390  8873 111  215 49.6% 0.095
MOT16-11 69.1% 76.2% 63.2% 80.2% 96.6%  75 44 24  7 266  1871  31   89 77.0% 0.083
OVERALL  56.2% 72.0% 46.1% 61.7% 96.3% 137 55 62 20 656 10744 142  304 58.8% 0.089
-------- EPOCH 15 --------


Iter 100. Loss: 0.011. Accuracy: 0.999. Recall: 0.999. Precision: 0.987
Iter 200. Loss: 0.012. Accuracy: 0.998. Recall: 0.999. Precision: 0.983
Iter 300. Loss: 0.010. Accuracy: 0.999. Recall: 0.999. Precision: 0.988
Iter 400. Loss: 0.010. Accuracy: 0.999. Recall: 0.999. Precision: 0.987

Tracking: MOT16-02
Tracks found: 86
Runtime for MOT16-02: 9.1 s.
Tracking: MOT16-11
Tracks found: 81
Runtime for MOT16-11: 10.3 s.
Runtime for all sequences: 19.4 s.
          IDF1   IDP   IDR  Rcll  Prcn  GT MT PT ML  FP    FN IDs   FM  MOTA  MOTP
MOT16-02 48.9% 69.4% 37.7% 52.2% 96.1%  62 11 38 13 390  8873 108  215 49.6% 0.095
MOT16-11 69.1% 76.2% 63.2% 80.2% 96.6%  75 44 24  7 266  1871  31   89 77.0% 0.083
OVERALL  56.5% 72.4% 46.3% 61.7% 96.3% 137 55 62 20 656 10744 139  304 58.8% 0.089


# Exercise submission

After executing the followinc cell the `Colab Notebooks/cv3dst_exercise/output` directory in your Google Drive should contain multiple `MOT16-XY.txt` files.

For the final submission you have to process the test sequences and upload the zipped prediction files to our server. See moodle for a guide how to upload the results.

Note that this time, you only have to evaluate three sequences!

In [20]:
best_ckpt = torch.load(osp.join(root_dir, 'output', 'best_ckpt.pth'))
assign_net.load_state_dict(best_ckpt)

tracker =  MPNTracker(assign_net=assign_net.eval(), obj_detect=None, patience=MAX_PATIENCE)
test_db = torch.load(osp.join(gnn_root_dir, 'data/preprocessed_data_test_2.pth'))
val_sequences = MOT16Sequences('MOT16-test', osp.join(root_dir, 'data/MOT16'), vis_threshold=0.)
run_tracker(val_sequences, db=test_db, tracker=tracker, output_dir=osp.join(root_dir, 'output'))

Tracking: MOT16-01
No GT evaluation data available.
Tracks found: 48
Runtime for MOT16-01: 4.7 s.
Writing predictions to: ./cv3dst_exercise/output\MOT16-01.txt
Tracking: MOT16-03
No GT evaluation data available.
Tracks found: 358
Runtime for MOT16-03: 18.6 s.
Writing predictions to: ./cv3dst_exercise/output\MOT16-03.txt
Tracking: MOT16-08
No GT evaluation data available.
Tracks found: 103
Runtime for MOT16-08: 6.7 s.
Writing predictions to: ./cv3dst_exercise/output\MOT16-08.txt
Tracking: MOT16-12
No GT evaluation data available.
Tracks found: 114
Runtime for MOT16-12: 9.2 s.
Writing predictions to: ./cv3dst_exercise/output\MOT16-12.txt
Runtime for all sequences: 39.2 s.
